In [1]:
import keras
from keras import backend as K
import numpy as np
import tensorflow as tf
from keras.models import Model
import ipynb.fs.full.Structural_Perturbations as SP
import ipynb.fs.full.MultiTest as MT
tf.logging.set_verbosity(tf.logging.ERROR)
import sys
from keras.layers import Input, Dense, Lambda, merge,add,concatenate
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.models import load_model

/home/uttaran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/uttaran/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Data has apparently already been downloaded and unpacked.


In [2]:
train_data,train_labels,eval_data,eval_labels = SP.load(data='cifar10')

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [ ]:
train_labels = keras.utils.to_categorical(train_labels, 10)
eval_labels = keras.utils.to_categorical(eval_labels, 10)

In [7]:
print(train_data.shape)

(50000, 32, 32, 3)


In [14]:
class MNIST:
    def __init__(self):
        self.build_model()
        
    def build_model(self):
        self.inputs = Input(shape=(32,32,3 ),name='imgs')
        self.labels = Input((10,),name='labels')
        inputs = self.inputs
        labels = self.labels

        conv1 = Conv2D(8, kernel_size=(3, 3),
                         activation='relu',
                         )(inputs)

        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(16, (3, 3), activation='relu')(pool1)

        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        pool2_flat = Flatten()(pool2)

        dense1 = Dense(128, activation='relu')(pool2_flat)
        dropout = Dropout(0.5)(dense1)

        logits = Dense(10)(dropout)

        self.predictions = Activation('softmax')(logits)

        g = merge([logits, pool2, labels], mode=lambda x: K.gradients(x[0] * x[2], x[1]), output_shape=(pool2.shape))
        cost = merge([labels, self.predictions], mode=lambda x: (-1) * K.sum(x[0] * K.log(x[1]), axis=1), output_shape=labels.shape)
        gb_grad = merge([cost, inputs], mode=lambda x: K.gradients(x[0], x[1]), output_shape=(inputs.shape))

In [15]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

m = MNIST()

model = Model(inputs=[m.inputs, m.labels], outputs=[m.predictions])

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/uttaran/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [5]:
model = load_model('./keras/models/mnist_test1.h5')

In [21]:
model.fit([train_data,train_labels],train_labels,
          batch_size=256,
          epochs=50,
          verbose=1)
model.save('./keras/models/cifar10_small.h5')
model.summary()

Epoch 1/50
50000/50000 [==============================] - 3s 53us/step - loss: 0.6865 - acc: 0.7412
Epoch 2/50
50000/50000 [==============================] - 3s 52us/step - loss: 0.6846 - acc: 0.7416
Epoch 3/50
50000/50000 [==============================] - 3s 52us/step - loss: 0.6792 - acc: 0.7434
Epoch 4/50
50000/50000 [==============================] - 3s 53us/step - loss: 0.6853 - acc: 0.7424
Epoch 5/50
50000/50000 [==============================] - 3s 53us/step - loss: 0.6837 - acc: 0.7412
Epoch 6/50
50000/50000 [==============================] - 3s 53us/step - loss: 0.6851 - acc: 0.7416
Epoch 7/50
50000/50000 [==============================] - 3s 52us/step - loss: 0.6790 - acc: 0.7450
Epoch 8/50
50000/50000 [==============================] - 3s 51us/step - loss: 0.6711 - acc: 0.7478
Epoch 9/50
50000/50000 [==============================] - 3s 52us/step - loss: 0.6798 - acc: 0.7459
Epoch 10/50
50000/50000 [==============================] - 3s 51us/step - loss: 0.6740 - acc: 0.7454

In [22]:
model.evaluate([eval_data,eval_labels],eval_labels)

10000/10000 [==============================] - 1s 74us/step


[1.2145355797767639, 0.6535]